# COMS 4995_002 Deep Learning Assignment 1
Due on Thursday, Feb 8, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Apoorv Purwar, ap3644

Member 2: Shreyas Mundhra, ssm2211

Format: layer_dimensions, alpha, batch_size, iters, drop_prob, reg_lambda, type, train_accuracy, test_accuracy, comment

Part 1:
    
Part 2:
    [X_train.shape[0], 512, 256, 128, 10], 0.001, 200, 10000, 0.3, 0.0, adam[0.9,0.999,1e-8], 44.57054423523343, 42.45939675174014, annealing
    [X_train.shape[0], 512, 256, 128, 10], 0.001, 200, 10000, 0.3, 0.0, adam[0.9,0.999,1e-8], 44.57054423523343, 42.45939675174014, without annealing
    [X_train.shape[0], 512, 256, 128, 10], 0.01, 200, 10000, 0.0, 0.0, rmsprop[0.9,1e-3], 75.85762187258189, 42.69141531322506, annealing
    [X_train.shape[0], 512, 256, 128, 10], 0.01, 600, 10000, 0.2, 0.2, rmsprop[0.9,1e-3], 34.04049522826928, 33.75870069605568, annealing, l2 reg
    [X_train.shape[0], 512, 512, 10], 0.01, 600, 10000, 0.0, 0.0, adam[0.9,0.99,1e-4], 53.391797781790046, 42.98143851508121, annealing
    [X_train.shape[0], 512, 512, 10], 0.01, 600, 20000, 0.0, 0.0, adam[0.9,0.99,1e-4], 77.97265927263348, 45.53364269141532, annealing
    [X_train.shape[0], 512, 512, 10], 0.01, 800, 20000, 0.0, 0.2, adam[0.9,0.99,1e-4], , without annealing
    [X_train.shape[0], 512, 512, 10], 0.01, 200, 20000, 0.0, 0.0, momentum[0.8], 72.5303069383544, 43.735498839907194, annealing
    [X_train.shape[0], 512, 512, 10], 0.001, 64, 20000, 0.0, 0.0, rmsprop[0.9,1e-3], 85.0657725045138, 50.17401392111369, without annealing
     [X_train.shape[0], 600, 600, 10], 0.001, 64, 20000, 0.0, 0.0, rmsprop[0.9,1e-3], 86.32318803198349, 50.29002320185615, without annealing
     [X_train.shape[0], 600, 600, 10], 0.01, 200, 35000, 0.5, 0.0, 86.76167139540883, 51.68213457076566, without annealing

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [20]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.num_layers = len(layer_dimensions) #Length of tuple is the number of layers
        self.parameters = {} 
        for i in range(1, self.num_layers):
            curr_layer_dim = layer_dimensions[i]
            prev_layer_dim = layer_dimensions[i-1]
            epsilon = np.sqrt(2.0 / (curr_layer_dim + prev_layer_dim))
            # Random Initialisation of weights
            weight = np.random.randn(curr_layer_dim, prev_layer_dim) * epsilon
            # Random Initialisation of bias
            bias = np.zeros((curr_layer_dim, 1)) + 0.01 # added to make sure relus fire in the start
            self.parameters[i] = [weight, bias]
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda  
        
    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        y = np.matmul(W,A) + b
        cache = [A,y,W]
        return y, cache
        

    def activationForward(self, A, activation="relu"):
        """¬
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        activation = self.relu(A)  # since no other activation fn exists
        return activation


    def relu(self, X):
        A = np.maximum(0, X)
        return A
            
    def dropout(self, A, prob):
        """
       :param A: 
       :param prob: drop prob
       :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        
        M = (np.random.rand(*A.shape) > prob)*1.0/(1-prob)
        A = A*M
        return A, M


    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = {}
        prev_activation = X 
        for i in range(1, self.num_layers-1):
            nodes = layer_dimensions[i]
            curr_activations = np.zeros((nodes,X.shape[1])) # Get the column count of X - Can we comment this?
            curr_activations, cache[i] = \
            self.affineForward(prev_activation, self.parameters[i][0], self.parameters[i][1])
            curr_activations = self.activationForward(curr_activations)
            
            # dropout
            curr_activations, M = self.dropout(curr_activations, self.drop_prob)
            cache[i] = [cache[i][0],cache[i][1],cache[i][2],M]    
            
            prev_activation = curr_activations
            
        
        nodes = layer_dimensions[self.num_layers-1]
        curr_activations = np.zeros((nodes,X.shape[1])) # Get the column count of X - Can we comment this?
        curr_activations, cache[self.num_layers-1] = \
        self.affineForward(prev_activation, self.parameters[self.num_layers-1][0], self.parameters[self.num_layers-1][1])
        
        AL = curr_activations

        return AL, cache

    def softmax(self, X):
        return np.exp(X) / np.sum(np.exp(X),axis=0) #changes input into probabilities

    def regularize(self, reg='l1'):
        reg_term = 0
        if reg == 'l1':
            for i in range(1,self.num_layers):
                reg_term = reg_term + self.reg_lambda*np.sum(np.absolute(self.parameters[i][0]))
        elif reg == 'l2':
            for i in range(1, self.num_layers):
                reg_term = reg_term + 0.5*self.reg_lambda*np.sum(np.square(self.parameters[i][0]))
        return reg_term
            
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        
        numberSamples = AL.shape[1]
        AL = AL - np.max(AL, axis=0, keepdims=True) #shifted values 
        AL_prob = self.softmax(AL)
        correct_label_prob = AL_prob[y, np.arange(numberSamples)]
        logProb = np.log(correct_label_prob)
        cost = -np.sum(logProb) / numberSamples
        # gradient of cost
        AL_prob[y, np.arange(numberSamples)] -= 1
        dAL = AL_prob/float(numberSamples)
        
        if self.reg_lambda > 0:
            # add regularization
            cost = cost + self.regularize(reg='l2')
            
        return cost, dAL
            

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A = cache[0]
        y = cache[1]
        W = cache[2]
                
        S = y.shape[1]
                
        dy = dA_prev
        dA = np.matmul(W.transpose(),dy)
        
        dW = np.matmul(dy, A.transpose())
        db = np.sum(dy, axis=1)
        db = db.reshape((len(db), 1))
        return dA, dW, db


    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        dx = self.relu_derivative(dA,cache[1])
        return dx

        
    def relu_derivative(self, dx, cached_x):
        nodes, S = cached_x.shape
        cached_x_derivative = np.zeros((nodes,S))       
        cached_x_derivative[cached_x > 0] = 1    
        dx = np.multiply(dx,cached_x_derivative)
        return dx


    def dropout_backward(self, dA, cache):
        dA = dA*cache[3]
        return dA

    
    def reg_derivative(self, gradients, reg='l1'):
        if reg == 'l1':
            for i in range(1, self.num_layers):
                dW = gradients[i][0]
                dW = dW + self.reg_lambda*np.sign(self.parameters[i][0])
                gradients[i][0] = dW
        elif reg == 'l2':
            for i in range(1, self.num_layers):
                dW = gradients[i][0]
                dW = dW + self.reg_lambda*self.parameters[i][0]
                gradients[i][0] = dW

        return gradients

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        
        S = len(Y)
        
        dA_prev, dW, db = self.affineBackward(dAL, cache[self.num_layers-1])
        gradients[self.num_layers-1] = [dW,db]
        
        for l in range(self.num_layers-2,0,-1):
            # dropout
            if self.drop_prob > 0:
                dA_prev = self.dropout_backward(dA_prev, cache[l])
                
            dA_prev = self.activationBackward(dA_prev, cache[l])
            dA, dW, db = self.affineBackward(dA_prev, cache[l])            
            gradients[l] = [dW,db]
            dA_prev = dA
           
#         # regularization
#         if self.reg_lambda > 0:
#             gradients = self.reg_derivative(gradients, reg='l2')
        
        return gradients  
    
    def momentum(self,gradients,m,iteration,beta=0.9):
        for l in range(1, self.num_layers):
            dW = gradients[l][0]
            db = gradients[l][1]
            if(iteration == 0):
                m[l] = [dW, db]
            else:
                m[l][0] = (1-beta)*dW - beta*m[l][0]
                m[l][1] = (1-beta)*db - beta*m[l][1]
            
            # momentum
            dW = m[l][0]
            db = m[l][1]
            gradients[l] = [dW, db]
            
        return gradients
    
    def rmsprop(self,gradients,s,iteration,beta=0.9,epsilon=1e-2):
        for l in range(1, self.num_layers):
            dW = gradients[l][0]
            db = gradients[l][1]
            if(iteration == 0):
                s[l] = [dW*dW, db*db]
            else:
                s[l][0] = (1-beta)*dW*dW + beta*s[l][0]
                s[l][1] = (1-beta)*db*db + beta*s[l][1]
            s1 = s[l][0]
            s2 = s[l][1]
            
            # rmsprop
            dW = dW/np.sqrt(s1 + epsilon)
            db = db/np.sqrt(s2 + epsilon)
            gradients[l] = [dW, db]
            
        return gradients, s

    def adam(self,gradients,m,s,iteration,beta1=0.9,beta2=0.999,epsilon=1e-2):
        for l in range(1, self.num_layers):
            dW = gradients[l][0]
            db = gradients[l][1]
            if(iteration == 0):
                m[l] = [dW, db]
                s[l] = [dW*dW, db*db]
            else:
                m[l][0] = (1-beta1)*dW - beta1*m[l][0]
                m[l][1] = (1-beta1)*db - beta1*m[l][1]
                s[l][0] = (1-beta2)*dW*dW + beta2*s[l][0]
                s[l][1] = (1-beta2)*db*db + beta2*s[l][1]
            s1 = s[l][0]
            s2 = s[l][1]
            m1 = m[l][0]
            m2 = m[l][1]
            
            # adam
            dW = m1/np.sqrt(s1 + epsilon)
            db = m2/np.sqrt(s2 + epsilon)
            gradients[l] = [dW, db]
            
        return gradients, m, s

    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for i in range(1, self.num_layers):
            self.parameters[i][0] = self.parameters[i][0] - alpha*gradients[i][0]
            self.parameters[i][1] = self.parameters[i][1] - alpha*gradients[i][1]

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        # rmsprop
        s = {}
        # momentum
        m = {}
        
        S = len(y)
        train_split_percent = 90
        
        num_train = int(train_split_percent*S/100)
        num_val = S - num_train
        
        X_train = X[:, :num_train]
        y_train = y[:num_train]
        X_val = X[:, num_train:]
        y_val = y[num_train:]
        
        X_batch, y_batch = self.get_batch(X_train,y_train,batch_size)
        numBatches = len(y_batch)
        
        j = 0
        for i in range(0, iters):
#             if (j+1%20) == 0:
#                 alpha = alpha*0.75
            # get minibatch
            X_mini = X_batch[i % numBatches]
            y_mini = y_batch[i % numBatches]

            # forward prop
            AL, cache = self.forwardPropagation(X_mini)

            # compute loss
            cost, dAL = self.costFunction(AL, y_mini)

            # compute gradients
            gradients = self.backPropagation(dAL, y_mini, cache)
            
            # update with adam
#             gradients, m, s = self.adam(gradients,m,s,i,beta1=0.9,beta2=0.99,epsilon=1e-4)
            
            # update with rmsprop
#             gradients, s = self.rmsprop(gradients,s,i,beta=0.9,epsilon=1e-3)

            # update with momentum
#             gradients = self.momentum(gradients,m,i,beta=0.8)
#             m = gradients

            # regularization
            if self.reg_lambda > 0:
                gradients = self.reg_derivative(gradients, reg='l2')

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)

            if i % print_every == 0:
                y_pred_train = self.predict(X_train)
                
                y_pred_val = self.predict(X_val)
                
                train_preds = y_pred_train
                train_accuracy = 0
                for i in range(0, len(y_train)):
                    if y_pred_train[y_train[i],i] == 1:
                        train_accuracy = train_accuracy + 1
                train_accuracy = train_accuracy*100.0/num_train
                
                val_accuracy = 0
                for i in range(0, len(y_val)):
                    if y_pred_val[y_val[i],i] == 1:
                        val_accuracy = val_accuracy + 1
                val_accuracy = val_accuracy*100.0/num_val
                
                # print cost, train and validation set accuracies
                j = j+1
                print("Iter -",j," : Loss, training accuracy, val accuracy: ", cost, train_accuracy, val_accuracy)
                
                
    def predict(self, X):
        """
        Make predictions for each sample
        """      
        drop_prob = self.drop_prob
        self.drop_prob = 0.0
        AL, cache = self.forwardPropagation(X)
        self.drop_prob = drop_prob
        
        S = AL.shape[1]
        y_pred = np.zeros(S, dtype=int)
        
        for j in range(0, S):
            y_pred[j] = np.argmax(AL[:,j])
        
        return one_hot(y_pred)

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        n = len(y)
        
        X_batch = []
        y_batch = []
        for start in range(0,n-batch_size,batch_size):
            X_batch.append(X[:,start:start+batch_size])
            y_batch.append(y[start:start+batch_size])
            
        return X_batch, y_batch

In [4]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [15]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [6]:
# Load the data
data_root_path = './cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'automobile': 1, 'deer': 4, 'dog': 5, 'frog': 6, 'cat': 3, 'truck': 9, 'ship': 8, 'airplane': 0, 'bird': 2}
Loaded 10000/17232
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [10]:
# layer_dimensions = [X_train.shape[0], 3, 10]  # including the input and output layers
# NN = NeuralNetwork(layer_dimensions)
# NN.train(X_train, y_train, alpha=0.0001,print_every=100)
layer_dimensions = [X_train.shape[0], 512, 512, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=10000, alpha=0.01, batch_size=128, print_every=100)

('Iter -', 1, ' : Loss, training accuracy, val accuracy: ', 2.3514135807250143, 10.994325509414496, 11.078886310904872)
('Iter -', 2, ' : Loss, training accuracy, val accuracy: ', 2.1787128823903745, 14.179778179004385, 14.559164733178655)
('Iter -', 3, ' : Loss, training accuracy, val accuracy: ', 2.1259471315731844, 23.71034304874903, 24.071925754060324)
('Iter -', 4, ' : Loss, training accuracy, val accuracy: ', 2.1171388013366945, 25.444931648181583, 24.94199535962877)
('Iter -', 5, ' : Loss, training accuracy, val accuracy: ', 2.0117988209803355, 26.78617487748259, 28.306264501160094)
('Iter -', 6, ' : Loss, training accuracy, val accuracy: ', 2.1568735545502169, 26.218725818932164, 28.65429234338747)
('Iter -', 7, ' : Loss, training accuracy, val accuracy: ', 2.0530858020060716, 27.237554810420427, 28.7122969837587)
('Iter -', 8, ' : Loss, training accuracy, val accuracy: ', 1.9746672437298833, 28.030693835439774, 30.220417633410673)
('Iter -', 9, ' : Loss, training accuracy, val

KeyboardInterrupt: 

layer_dimensions = [X_train.shape[0], 400, 400, 10] 
if (j+1%50) == 0:
                alpha = alpha/10

Iter - 1  : Loss, training accuracy, val accuracy:  2.44146032806 10.733333333333333 10.7
Iter - 2  : Loss, training accuracy, val accuracy:  2.04368446456 24.07111111111111 23.92
Iter - 3  : Loss, training accuracy, val accuracy:  2.00188142378 28.384444444444444 28.2
Iter - 4  : Loss, training accuracy, val accuracy:  2.18635498903 31.433333333333334 31.5
Iter - 5  : Loss, training accuracy, val accuracy:  1.94207102157 33.74444444444445 33.2
Iter - 6  : Loss, training accuracy, val accuracy:  1.73651363821 35.40888888888889 35.7
Iter - 7  : Loss, training accuracy, val accuracy:  2.07354837025 33.63777777777778 33.56
Iter - 8  : Loss, training accuracy, val accuracy:  1.95194533663 34.471111111111114 34.62
Iter - 9  : Loss, training accuracy, val accuracy:  2.07356046653 37.806666666666665 37.56
Iter - 10  : Loss, training accuracy, val accuracy:  1.67569080264 37.715555555555554 38.12
Iter - 11  : Loss, training accuracy, val accuracy:  1.7786504777 37.69111111111111 38.0
Iter - 12  : Loss, training accuracy, val accuracy:  1.90228663986 37.324444444444445 37.24
Iter - 13  : Loss, training accuracy, val accuracy:  1.81155344849 39.333333333333336 38.58
Iter - 14  : Loss, training accuracy, val accuracy:  1.81668963683 38.09777777777778 37.7
Iter - 15  : Loss, training accuracy, val accuracy:  1.65419163901 38.455555555555556 39.04
Iter - 16  : Loss, training accuracy, val accuracy:  1.55458309623 39.33555555555556 39.24
Iter - 17  : Loss, training accuracy, val accuracy:  2.18991704386 40.44888888888889 40.34
Iter - 18  : Loss, training accuracy, val accuracy:  1.95713638464 36.05111111111111 35.64
Iter - 19  : Loss, training accuracy, val accuracy:  1.61187302584 41.39333333333333 40.78
Iter - 20  : Loss, training accuracy, val accuracy:  1.89105380614 40.553333333333335 39.96
Iter - 21  : Loss, training accuracy, val accuracy:  1.6095329067 41.42666666666667 41.02
Iter - 22  : Loss, training accuracy, val accuracy:  1.60520797047 40.68888888888889 40.9
Iter - 23  : Loss, training accuracy, val accuracy:  1.69769797414 38.973333333333336 38.78
Iter - 24  : Loss, training accuracy, val accuracy:  1.76547442066 41.593333333333334 40.68
Iter - 25  : Loss, training accuracy, val accuracy:  1.8544110434 41.653333333333336 41.5
Iter - 26  : Loss, training accuracy, val accuracy:  1.73883525466 41.833333333333336 41.4
Iter - 27  : Loss, training accuracy, val accuracy:  1.77102964349 41.95777777777778 41.74
Iter - 28  : Loss, training accuracy, val accuracy:  1.42637753188 42.27333333333333 41.24
Iter - 29  : Loss, training accuracy, val accuracy:  1.45903627197 41.75333333333333 41.36
Iter - 30  : Loss, training accuracy, val accuracy:  1.47681544572 44.004444444444445 43.48
Iter - 31  : Loss, training accuracy, val accuracy:  1.87098673539 44.66444444444444 43.72
Iter - 32  : Loss, training accuracy, val accuracy:  1.68313372423 43.3 42.44
Iter - 33  : Loss, training accuracy, val accuracy:  1.67122354619 42.11333333333334 41.66
Iter - 34  : Loss, training accuracy, val accuracy:  1.66710783799 43.51555555555556 42.74
Iter - 35  : Loss, training accuracy, val accuracy:  1.50972828603 42.62888888888889 41.56
Iter - 36  : Loss, training accuracy, val accuracy:  1.64361718129 43.742222222222225 42.48
Iter - 37  : Loss, training accuracy, val accuracy:  1.46363720167 44.42 43.48
Iter - 38  : Loss, training accuracy, val accuracy:  1.55401218323 43.14888888888889 42.56
Iter - 39  : Loss, training accuracy, val accuracy:  1.69097204928 44.07333333333333 42.78
Iter - 40  : Loss, training accuracy, val accuracy:  1.59478691477 45.26222222222222 44.22
Iter - 41  : Loss, training accuracy, val accuracy:  1.75819273744 43.37555555555556 42.3
Iter - 42  : Loss, training accuracy, val accuracy:  1.56366021784 43.98 42.92
Iter - 43  : Loss, training accuracy, val accuracy:  1.88991499727 44.07555555555555 43.08
Iter - 44  : Loss, training accuracy, val accuracy:  1.42292337774 44.85777777777778 44.42
Iter - 45  : Loss, training accuracy, val accuracy:  1.37594318592 44.29333333333334 43.12
Iter - 46  : Loss, training accuracy, val accuracy:  1.41177538881 44.86666666666667 44.1
Iter - 47  : Loss, training accuracy, val accuracy:  1.2053165101 45.90222222222222 45.3
Iter - 48  : Loss, training accuracy, val accuracy:  1.20435805293 43.68222222222222 42.46
Iter - 49  : Loss, training accuracy, val accuracy:  1.43970085773 44.964444444444446 44.02
Iter - 50  : Loss, training accuracy, val accuracy:  1.38235931503 40.41555555555556 39.7
Iter - 51  : Loss, training accuracy, val accuracy:  1.44756407969 44.24444444444445 43.6
Iter - 52  : Loss, training accuracy, val accuracy:  1.9659051155 45.70444444444445 44.58
Iter - 53  : Loss, training accuracy, val accuracy:  1.75350059995 45.95777777777778 44.44
Iter - 54  : Loss, training accuracy, val accuracy:  1.56627904598 47.568888888888885 46.04
Iter - 55  : Loss, training accuracy, val accuracy:  1.90079785668 46.355555555555554 45.16
Iter - 56  : Loss, training accuracy, val accuracy:  1.85225052992 44.98444444444444 43.84
Iter - 57  : Loss, training accuracy, val accuracy:  1.91424211646 45.54666666666667 44.96
Iter - 58  : Loss, training accuracy, val accuracy:  1.51412655108 44.58444444444444 44.62
Iter - 59  : Loss, training accuracy, val accuracy:  1.69416287598 45.51111111111111 44.56
Iter - 60  : Loss, training accuracy, val accuracy:  1.30677446984 47.702222222222225 45.8
Iter - 61  : Loss, training accuracy, val accuracy:  1.38051377098 46.455555555555556 45.18
Iter - 62  : Loss, training accuracy, val accuracy:  1.27817202624 45.98222222222222 44.72
Iter - 63  : Loss, training accuracy, val accuracy:  1.54186271326 45.635555555555555 44.26
Iter - 64  : Loss, training accuracy, val accuracy:  1.5438762561 47.91777777777778 46.44
Iter - 65  : Loss, training accuracy, val accuracy:  1.39112605061 47.69777777777778 46.26
Iter - 66  : Loss, training accuracy, val accuracy:  1.22741366993 47.684444444444445 46.64
Iter - 67  : Loss, training accuracy, val accuracy:  1.79225958645 47.98222222222222 46.62
Iter - 68  : Loss, training accuracy, val accuracy:  1.54878430873 48.382222222222225 46.48
Iter - 69  : Loss, training accuracy, val accuracy:  1.79299760528 48.12222222222222 46.76
Iter - 70  : Loss, training accuracy, val accuracy:  1.44048610354 48.03333333333333 46.96
Iter - 71  : Loss, training accuracy, val accuracy:  1.30484932521 47.55777777777778 45.88
Iter - 72  : Loss, training accuracy, val accuracy:  1.50550837904 49.93555555555555 47.64
Iter - 73  : Loss, training accuracy, val accuracy:  1.70241748241 46.29333333333334 45.12
Iter - 74  : Loss, training accuracy, val accuracy:  1.52762987348 49.477777777777774 47.44
Iter - 75  : Loss, training accuracy, val accuracy:  1.35255465374 47.07333333333333 45.36
Iter - 76  : Loss, training accuracy, val accuracy:  1.0955502084 46.004444444444445 44.42
Iter - 77  : Loss, training accuracy, val accuracy:  1.78851390465 48.48888888888889 46.44
Iter - 78  : Loss, training accuracy, val accuracy:  1.25808027476 48.49333333333333 46.94
Iter - 79  : Loss, training accuracy, val accuracy:  1.38979317373 48.40888888888889 46.62
Iter - 80  : Loss, training accuracy, val accuracy:  1.42829389768 49.715555555555554 46.88
Iter - 81  : Loss, training accuracy, val accuracy:  1.59232738895 48.28666666666667 46.14
Iter - 82  : Loss, training accuracy, val accuracy:  1.75743690579 47.824444444444445 45.36
Iter - 83  : Loss, training accuracy, val accuracy:  1.47744509043 48.42 46.48
Iter - 84  : Loss, training accuracy, val accuracy:  1.47370424767 48.526666666666664 47.44
Iter - 85  : Loss, training accuracy, val accuracy:  1.45841012545 49.33777777777778 47.66
Iter - 86  : Loss, training accuracy, val accuracy:  1.36990435669 49.824444444444445 48.06
Iter - 87  : Loss, training accuracy, val accuracy:  1.46146524721 50.035555555555554 47.58
Iter - 88  : Loss, training accuracy, val accuracy:  1.28815780111 48.75111111111111 47.54
Iter - 89  : Loss, training accuracy, val accuracy:  1.314553076 48.08222222222222 46.78
Iter - 90  : Loss, training accuracy, val accuracy:  1.56415913984 49.66222222222222 47.4
Iter - 91  : Loss, training accuracy, val accuracy:  1.42180077064 49.18222222222222 47.02
Iter - 92  : Loss, training accuracy, val accuracy:  1.34811620935 49.068888888888885 47.16
Iter - 93  : Loss, training accuracy, val accuracy:  1.66548980377 49.92888888888889 47.82
Iter - 94  : Loss, training accuracy, val accuracy:  1.40904177043 48.79333333333334 46.08
Iter - 95  : Loss, training accuracy, val accuracy:  1.49923030442 50.77777777777778 47.9
Iter - 96  : Loss, training accuracy, val accuracy:  1.32606640219 48.21333333333333 46.04
Iter - 97  : Loss, training accuracy, val accuracy:  1.49399061959 49.766666666666666 47.84
Iter - 98  : Loss, training accuracy, val accuracy:  1.33230507242 50.215555555555554 48.02
Iter - 99  : Loss, training accuracy, val accuracy:  1.27471341741 49.63777777777778 47.8
Iter - 100  : Loss, training accuracy, val accuracy:  1.29687341586 46.775555555555556 45.6
Iter - 101  : Loss, training accuracy, val accuracy:  1.22522657352 50.513333333333335 47.94
Iter - 102  : Loss, training accuracy, val accuracy:  1.41175838211 50.595555555555556 48.52
Iter - 103  : Loss, training accuracy, val accuracy:  1.32216031724 50.38444444444445 47.86
Iter - 104  : Loss, training accuracy, val accuracy:  1.46303287434 50.38444444444445 47.88
Iter - 105  : Loss, training accuracy, val accuracy:  1.01979675294 51.04222222222222 48.68
Iter - 106  : Loss, training accuracy, val accuracy:  1.1049326242 51.22 49.12
Iter - 107  : Loss, training accuracy, val accuracy:  1.14325530218 49.977777777777774 47.22
Iter - 108  : Loss, training accuracy, val accuracy:  1.41086484351 51.30444444444444 48.3
Iter - 109  : Loss, training accuracy, val accuracy:  1.12775289279 50.30222222222222 47.26
Iter - 110  : Loss, training accuracy, val accuracy:  1.58760645066 50.31111111111111 47.66
Iter - 111  : Loss, training accuracy, val accuracy:  1.36885523759 51.873333333333335 49.18
Iter - 112  : Loss, training accuracy, val accuracy:  1.28283915679 51.25111111111111 48.44
Iter - 113  : Loss, training accuracy, val accuracy:  1.2286260388 51.41555555555556 49.0
Iter - 114  : Loss, training accuracy, val accuracy:  1.3667402492 51.977777777777774 49.6
Iter - 115  : Loss, training accuracy, val accuracy:  1.43743999649 52.24444444444445 48.98
Iter - 116  : Loss, training accuracy, val accuracy:  1.66923978428 49.82222222222222 47.04
Iter - 117  : Loss, training accuracy, val accuracy:  1.33126731042 52.49333333333333 49.24
Iter - 118  : Loss, training accuracy, val accuracy:  1.36794875985 51.64888888888889 48.46
Iter - 119  : Loss, training accuracy, val accuracy:  1.51826228679 52.60888888888889 49.72
Iter - 120  : Loss, training accuracy, val accuracy:  1.58485819304 51.946666666666665 48.88
Iter - 121  : Loss, training accuracy, val accuracy:  1.26596125364 52.11333333333334 48.88
Iter - 122  : Loss, training accuracy, val accuracy:  1.47683673643 52.282222222222224 48.96
Iter - 123  : Loss, training accuracy, val accuracy:  1.12693038462 52.37777777777778 48.46
Iter - 124  : Loss, training accuracy, val accuracy:  1.36007717441 52.553333333333335 49.28
Iter - 125  : Loss, training accuracy, val accuracy:  1.39299061073 51.144444444444446 47.86
Iter - 126  : Loss, training accuracy, val accuracy:  1.50519721014 51.135555555555555 47.2
Iter - 127  : Loss, training accuracy, val accuracy:  1.27471846918 53.53333333333333 50.66
Iter - 128  : Loss, training accuracy, val accuracy:  1.33136928233 50.64222222222222 48.0
Iter - 129  : Loss, training accuracy, val accuracy:  1.15669033574 53.202222222222225 50.04
Iter - 130  : Loss, training accuracy, val accuracy:  1.55304838644 52.406666666666666 49.36
Iter - 131  : Loss, training accuracy, val accuracy:  1.23251917798 53.60666666666667 49.56
Iter - 132  : Loss, training accuracy, val accuracy:  1.29273861118 52.21111111111111 49.96
Iter - 133  : Loss, training accuracy, val accuracy:  1.37053974833 50.69555555555556 47.5
Iter - 134  : Loss, training accuracy, val accuracy:  1.36766788287 50.11333333333334 47.62
Iter - 135  : Loss, training accuracy, val accuracy:  1.27913653854 50.897777777777776 47.58
Iter - 136  : Loss, training accuracy, val accuracy:  1.66574830838 52.80222222222222 49.52
Iter - 137  : Loss, training accuracy, val accuracy:  1.12144202359 52.68222222222222 49.1
Iter - 138  : Loss, training accuracy, val accuracy:  1.49781903939 52.715555555555554 49.68
Iter - 139  : Loss, training accuracy, val accuracy:  1.52057331525 51.85777777777778 49.02
Iter - 140  : Loss, training accuracy, val accuracy:  1.57637153294 51.995555555555555 48.54
Iter - 141  : Loss, training accuracy, val accuracy:  1.33481059339 54.38666666666666 50.48
Iter - 142  : Loss, training accuracy, val accuracy:  1.09551036915 52.833333333333336 48.94
Iter - 143  : Loss, training accuracy, val accuracy:  1.33266717039 50.73777777777778 47.12
Iter - 144  : Loss, training accuracy, val accuracy:  1.02842661041 52.83777777777778 50.06
Iter - 145  : Loss, training accuracy, val accuracy:  1.02710632645 53.65555555555556 48.92
Iter - 146  : Loss, training accuracy, val accuracy:  1.04340136876 54.76888888888889 50.66
Iter - 147  : Loss, training accuracy, val accuracy:  1.23798911652 53.35777777777778 50.02
Iter - 148  : Loss, training accuracy, val accuracy:  1.69754839139 54.26222222222222 50.5
Iter - 149  : Loss, training accuracy, val accuracy:  1.06179587446 53.40222222222222 50.34
Iter - 150  : Loss, training accuracy, val accuracy:  0.970046335306 53.45333333333333 49.86
Iter - 151  : Loss, training accuracy, val accuracy:  1.08052386616 52.964444444444446 48.5
Iter - 152  : Loss, training accuracy, val accuracy:  0.961560542907 53.224444444444444 49.04
Iter - 153  : Loss, training accuracy, val accuracy:  1.44520760387 53.54888888888889 49.84
Iter - 154  : Loss, training accuracy, val accuracy:  1.30256508886 53.44444444444444 49.6
Iter - 155  : Loss, training accuracy, val accuracy:  1.29809901117 53.126666666666665 49.82
Iter - 156  : Loss, training accuracy, val accuracy:  1.25246796986 54.94888888888889 51.42
Iter - 157  : Loss, training accuracy, val accuracy:  1.14413038656 53.522222222222226 49.48
Iter - 158  : Loss, training accuracy, val accuracy:  1.12016892881 53.87111111111111 50.9
Iter - 159  : Loss, training accuracy, val accuracy:  1.22237439062 54.333333333333336 49.52
Iter - 160  : Loss, training accuracy, val accuracy:  1.35548525577 53.01111111111111 49.92
Iter - 161  : Loss, training accuracy, val accuracy:  1.24299329469 54.77111111111111 50.62
Iter - 162  : Loss, training accuracy, val accuracy:  1.30014665884 52.22 48.04
Iter - 163  : Loss, training accuracy, val accuracy:  1.33193116987 54.382222222222225 49.9
Iter - 164  : Loss, training accuracy, val accuracy:  1.43694042819 51.83777777777778 47.9
Iter - 165  : Loss, training accuracy, val accuracy:  1.39658214914 53.63777777777778 49.22
Iter - 166  : Loss, training accuracy, val accuracy:  1.50251164354 53.333333333333336 48.78
Iter - 167  : Loss, training accuracy, val accuracy:  1.34405515799 54.028888888888886 49.72
Iter - 168  : Loss, training accuracy, val accuracy:  0.891654104708 54.78888888888889 51.0
Iter - 169  : Loss, training accuracy, val accuracy:  1.53831072577 54.44444444444444 50.06
Iter - 170  : Loss, training accuracy, val accuracy:  1.22336252316 54.23555555555556 49.78
Iter - 171  : Loss, training accuracy, val accuracy:  1.43290403754 54.71111111111111 50.04
Iter - 172  : Loss, training accuracy, val accuracy:  1.59756656472 55.08444444444444 50.74
Iter - 173  : Loss, training accuracy, val accuracy:  1.45481111149 55.45111111111111 50.76
Iter - 174  : Loss, training accuracy, val accuracy:  1.16173697146 52.864444444444445 47.94
Iter - 175  : Loss, training accuracy, val accuracy:  1.54983433796 54.48222222222222 50.68
Iter - 176  : Loss, training accuracy, val accuracy:  1.22061704427 54.42444444444445 49.96
Iter - 177  : Loss, training accuracy, val accuracy:  1.50054273479 54.80888888888889 51.62
Iter - 178  : Loss, training accuracy, val accuracy:  1.21910162371 55.446666666666665 51.08
Iter - 179  : Loss, training accuracy, val accuracy:  1.00820851208 54.346666666666664 49.92
Iter - 180  : Loss, training accuracy, val accuracy:  1.51468366346 53.2 48.4
Iter - 181  : Loss, training accuracy, val accuracy:  1.23756636696 55.75111111111111 51.5
Iter - 182  : Loss, training accuracy, val accuracy:  1.1164545696 56.14222222222222 50.62

In [14]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [15]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
np.set_printoptions(threshold=np.nan)
print(loaded_y.shape)
loaded_y[:10]

(10, 10000)


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

## Part 2: Improving the performance

In [ ]:
def reflect_image_horizontal(image_arr):
    image_arr = image_arr.reshape((32,32,3))
    flip_image_arr = np.fliplr(image_arr)
    return flip_image_arr

In [ ]:
def augment_data(X,y,step, label_file):
    S = X.shape[1]
    j = S + 1
    for i in range(0,S,step):
        flipped_X = reflect_image_horizontal(X[:,i])
        flipped_y = y[i]
        id2label, label2id = get_label_mapping(label_file)
        directory = './aug_cifar/'
        file_name = str(j) + '_' + id2label[flipped_y] + '.png'
        scipy.misc.imsave(directory + file_name, flipped_X)
        j = j + 1

In [ ]:
# augment_data(X_train, y_train, 2872, data_root_path + 'labels.txt')

In [21]:
layer_dimensions = [X_train.shape[0], 600, 600, 10]  # including the input and output layers
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0.5, reg_lambda=0.0)
NN2.train(X_train, y_train, iters=35000, alpha=0.01, batch_size=200, print_every=100)

('Iter -', 1, ' : Loss, training accuracy, val accuracy: ', 2.7128631818472217, 11.045911787464535, 11.658932714617169)
('Iter -', 2, ' : Loss, training accuracy, val accuracy: ', 2.1729584548344389, 28.49497033789012, 28.48027842227378)
('Iter -', 3, ' : Loss, training accuracy, val accuracy: ', 2.1328592408502516, 30.01031725561001, 30.68445475638051)
('Iter -', 4, ' : Loss, training accuracy, val accuracy: ', 2.0367638812183175, 31.55790559711117, 32.59860788863109)
('Iter -', 5, ' : Loss, training accuracy, val accuracy: ', 1.9526163446796758, 33.49239102398762, 34.28074245939675)
('Iter -', 6, ' : Loss, training accuracy, val accuracy: ', 1.9886231297520869, 34.25328862522569, 35.092807424593964)
('Iter -', 7, ' : Loss, training accuracy, val accuracy: ', 2.0029167692904535, 34.81428939901986, 36.31090487238979)
('Iter -', 8, ' : Loss, training accuracy, val accuracy: ', 1.8540411983200165, 35.84601496002063, 36.54292343387471)
('Iter -', 9, ' : Loss, training accuracy, val accura

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

Write down results for Part 2 here:
...

In [ ]:
import numpy as np
a = np.array([0.7,0.8,0.6,0.3]).reshape(2,2)
# = np.arange(2*4*2).reshape((2,4,2))
# = np.arange(2*2*4).reshape((2,2,2))
#print("A = ", (np.random.rand(*a.shape)<0.3)/0.3)
#print("B = ", b)

#print("Dot = ", np.dot(a,b))

#print("Matmul = ", np.matmul(a,b))

#np.random.seed(0);
#np.random.rand(4)
#10*np.random.uniform()
#np.random.normal(0,2,(10,10))


In [ ]:
import numpy as np
bias = (np.random.uniform()/10)*np.ones((3,1)) 
# Random Initialisation of weights
weight = np.random.normal(0,10*np.random.uniform(),(3, 5)) 
param = [weight, bias]
print("weight", weight)
print("bias", bias)
print("param",param)